# Modeling a Zombie Apocalypse with ODEs

## Credits
* Zombie Apocalypse example by [Munz et al. 2009](https://mysite.science.uottawa.ca/rsmith43/Zombies.pdf)
* SciPy implementation by [Christopher Campo](https://scipy-cookbook.readthedocs.io/items/Zombie_Apocalypse_ODEINT.html)
* Jupyter Notebook interactivity added by Alex Lubbock

## Background

This example demonstrates how to solve a system of first order ODEs using SciPy. Note that a Nth order equation can also be solved using SciPy by transforming it into a system of first order equations. In a this lighthearted example, a system of ODEs can be used to model a "zombie invasion", using the equations specified in Munz et al. 2009.

![Zombie ODE model diagram](model-diagram.png)

The system is given as:

dS/dt = P - BSZ - dS dZ/dt = BSZ + GR - ASZ dR/dt = dS + ASZ - GR

with the following notations:

    S: the number of susceptible victims
    Z: the number of zombies
    R: the number of people "killed"
    P: the population birth rate
    d: the chance of a natural death
    B: the chance the "zombie disease" is transmitted (an alive person becomes a zombie)
    G: the chance a dead person is resurrected into a zombie
    A: the chance a zombie is totally destroyed

This involves solving a system of first order ODEs given by: dy/dt = f(y, t)

Where y = [S, Z, R].

## How to run this notebook

Select the **Kernel** menu at the top of the page, then select **Restart & Run All**. After a few seconds, a plot should appear at the bottom of the page. Adjust the slider bars to run a new ODE simulation and update the plot.

In [ ]:
# Enable interactive plots in Jupyter Notebook
%matplotlib notebook

# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from scipy.integrate import odeint

# Default parameter values

P = 0      # birth rate
d = 0.0001  # natural death percent (per day)
B = 0.0095  # transmission percent  (per day)
G = 0.0001  # resurect percent (per day)
A = 0.0001  # destroy percent  (per day)

# Solve the system dy/dt = f(y, t)
def f(y, t):
     Si, Zi, Ri = y

     # the model equations (see Munz et al. 2009)
     f0 = P - B*Si*Zi - d*Si
     f1 = B*Si*Zi + G*Ri - A*Si*Zi
     f2 = d*Si + A*Si*Zi - G*Ri
     return [f0, f1, f2]

# Initial conditions
S0 = 500.              # initial population
Z0 = 0                 # initial zombie population
R0 = 0                 # initial death population
y0 = [S0, Z0, R0]      # initial condition vector
t  = np.linspace(0, 5., 1000)   # time grid

# Solve the ODEs
soln = odeint(f, y0, t)
S = soln[:, 0]
Z = soln[:, 1]
R = soln[:, 2]

# Plot results
fig, ax = plt.subplots()
l1, = plt.plot(t, S, label='Living')
l2, = plt.plot(t, Z, label='Zombies')
plt.xlabel('Days from outbreak')
plt.ylabel('Population')
plt.title('Zombie Apocalypse')
plt.legend(loc=0)

# Adjust the main plot to make room for the sliders
plt.subplots_adjust(bottom=0.25)

# Add interactivity
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03])
s0_slider = Slider(
    ax=axfreq,
    label='S0: Init. human pop.',
    valmin=0,
    valmax=1000,
    valinit=S0,
)
ax_z0 = plt.axes([0.25, 0.05, 0.65, 0.03])
z0_slider = Slider(
    ax=ax_z0,
    label='Z0: Init. zombie pop.',
    valmin=0,
    valmax=1000,
    valinit=Z0,
)

# Callback function to re-plot new initial conditions
def run_ode(_):
    y0 = [s0_slider.val, z0_slider.val, R0]     # initial condition vector
    soln = odeint(f, y0, t)
    S = soln[:, 0]
    Z = soln[:, 1]
    R = soln[:, 2]
    l1.set_ydata(S)
    l2.set_ydata(Z)
    fig.canvas.draw_idle()

s0_slider.on_changed(run_ode)
z0_slider.on_changed(run_ode)